<a href="https://colab.research.google.com/github/kimheeseo/LDPC_study/blob/main/tensorflowbasic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 : https://github.com/neowizard2018/neowizard 

In [1]:
#30강
!pip uninstall tensorflow
#tensorflow 삭제

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [2]:
!pip install tensorflow==1.15 # 1.15 버전 Tensorflow 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 20 kB/s 
     |████████████████████████████████| 3.8 MB 38.1 MB/s 
     |████████████████████████████████| 50 kB 9.2 MB/s 
     |████████████████████████████████| 503 kB 60.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e75692f9060e151483306e91048a4fbeda1fbeaa7d60159ba7331c152d3033e0
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting un

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
import numpy as np

In [5]:
import pandas as pd
loaded_data = pd.read_csv('/content/drive/MyDrive/data/data2/diabetes.csv', sep=',')
loaded_data.head()

,-0.294118,0.487437,0.180328,-0.292929,0,0.00149028,-0.53117,-0.0333333,0.1
0,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1
1,-0.058824,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0
2,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1
3,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0
4,-0.411765,0.165829,0.213115,0.000000,0.000000,-0.236960,-0.894962,-0.700000,1


In [6]:
x_data = loaded_data.iloc[ :, 0:-1]
t_data = loaded_data.iloc[ :, [-1]]

print("loaded_data = ", loaded_data.shape)
print("x_data = ", x_data.shape, ", t_data = ", t_data.shape)

#error : (slice(None, None, None), slice(0, -1, None))' is an invalid key
#sol : iloc 출처 : https://stackoverflow.com/questions/55291667/getting-typeerror-slicenone-none-none-0-is-an-invalid-key/71027973#71027973

loaded_data =  (758, 9)
x_data =  (758, 8) , t_data =  (758, 1)


In [7]:
X = tf.placeholder(tf.float32, [None, 8])  # 8개 입력노드
T = tf.placeholder(tf.float32, [None, 1])  # 1개 정답노드

W = tf.Variable(tf.random_normal([8, 1]))  # 8X1 가중치노드
b = tf.Variable(tf.random_normal([1]))     # 1개 바이어스 노드

In [8]:
z = tf.matmul(X, W) + b  # 선형회귀 값 z

y = tf.sigmoid(z)    # 시그모이드로 계산 값

# 손실함수는 Cross-Entropy 
loss = -tf.reduce_mean( T*tf.log(y) + (1-T)*tf.log(1-y) ) 

In [9]:
learning_rate = 0.01    # 학습율

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [10]:
print("train값", train)

train값 name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



In [11]:
# 정확성 검사, True if y > 0.5 else False

# y shape 은 (759X8) (8X1) = 759X1, 즉 759개의 모든 데이터에 대해 y > 0.5 검사함
# 그러므로 predicted 또한 759 개의 1 또는 0 을 가짐
predicted = tf.cast(y > 0.5, dtype=tf.float32)  


# predicted 와 T 같으면 True 를 리턴하므로 cast 에 의해서 1로 강제 변환, 그렇지 않으면 0으로 변환해서
# accuracy 는 총 759 개의 1 또는 0 을 가짐. 이것을 전체 갯수 759로 나눔
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype=tf.float32))

In [15]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(20001):
        loss_val, _ = sess.run([loss, train], feed_dict={X: x_data, T: t_data})    
        
        
        if step % 500 == 0:
            print("step = ", step, ", loss_val = ", loss_val)        
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X: x_data, T: t_data})
    
    
    print("\ny_val.shape = ", y_val.shape, ", predicted_val = ", predicted_val.shape)    
    print("\nAccuracy = ", accuracy_val)

step =  0 , loss_val =  0.8806674
step =  500 , loss_val =  0.78304803
step =  1000 , loss_val =  0.7251702
step =  1500 , loss_val =  0.6781905
step =  2000 , loss_val =  0.6402506
step =  2500 , loss_val =  0.6098406
step =  3000 , loss_val =  0.5855654
step =  3500 , loss_val =  0.566197
step =  4000 , loss_val =  0.5507017
step =  4500 , loss_val =  0.53824174
step =  5000 , loss_val =  0.52815545
step =  5500 , loss_val =  0.5199299
step =  6000 , loss_val =  0.5131703
step =  6500 , loss_val =  0.50757366
step =  7000 , loss_val =  0.5029066
step =  7500 , loss_val =  0.49898842
step =  8000 , loss_val =  0.49567825
step =  8500 , loss_val =  0.4928652
step =  9000 , loss_val =  0.49046174
step =  9500 , loss_val =  0.48839772
step =  10000 , loss_val =  0.48661682
step =  10500 , loss_val =  0.48507348
step =  11000 , loss_val =  0.4837305
step =  11500 , loss_val =  0.48255724
step =  12000 , loss_val =  0.4815286
step =  12500 , loss_val =  0.48062354
step =  13000 , loss_val 

In [ ]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(20001):
        loss_val, _ = sess.run([loss, train], feed_dict={X: x_data, T: t_data})    
        
        
        if step % 500 == 0:
            print("step = ", step, ", loss_val = ", loss_val)        
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X: x_data, T: t_data})

In [19]:
#29강
import tensorflow as tf
import numpy as np

In [25]:
import pandas as pd
loaded_data2 = pd.read_csv('/content/drive/MyDrive/data/data2/data-01.csv', sep=',')
loaded_data2.head()

,73,80,75,152
0,93,88,93,185
1,89,91,90,180
2,96,98,100,196
3,73,66,70,142
4,53,46,55,101


In [26]:
x_data = loaded_data2.iloc[ :, 0:-1]
t_data = loaded_data2.iloc[ :, [-1]]

print("x_data.shape = ", x_data.shape)
print("t_data.shape = ", t_data.shape)

x_data.shape =  (24, 3)
t_data.shape =  (24, 1)


In [41]:
W = tf.Variable(tf.random_normal([3, 1]))  # 가중치 노드
b = tf.Variable(tf.random_normal([1]))     # 바이어스 노드

X = tf.placeholder(tf.float32, [None, 3])  # None 은 총 데이터 갯수
T = tf.placeholder(tf.float32, [None, 1])  # 정답데이터 노드
y = tf.matmul(X, W) + b  # 현재 X, W, b, 를 바탕으로 계산된 값

loss = tf.reduce_mean(tf.square(y - T))  # MSE 손실함수 정의
learning_rate = 1e-5    # 학습율

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [28]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(8001):
      
        loss_val, y_val, _ = sess.run([loss, y, train], feed_dict={X: x_data, T: t_data})    
        
        
        if step % 400 == 0:
            print("step = ", step, ", loss_val = ", loss_val)             
    
    print("\nPrediction is ", sess.run(y, feed_dict={X: [ [100, 98, 81] ]}))  

step =  0 , loss_val =  142.92792
step =  400 , loss_val =  20.363752
step =  800 , loss_val =  17.787987
step =  1200 , loss_val =  15.692184
step =  1600 , loss_val =  13.985924
step =  2000 , loss_val =  12.596133
step =  2400 , loss_val =  11.463623
step =  2800 , loss_val =  10.54041
step =  3200 , loss_val =  9.78761
step =  3600 , loss_val =  9.173551
step =  4000 , loss_val =  8.672594
step =  4400 , loss_val =  8.263786
step =  4800 , loss_val =  7.930136
step =  5200 , loss_val =  7.6577644
step =  5600 , loss_val =  7.4353867
step =  6000 , loss_val =  7.2538095
step =  6400 , loss_val =  7.1055255
step =  6800 , loss_val =  6.9843917
step =  7200 , loss_val =  6.8854537
step =  7600 , loss_val =  6.8046374
step =  8000 , loss_val =  6.7385864

Prediction is  [[180.14014]]


In [43]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(801):
          
        loss_val, y_val, _ = sess.run([loss, y, train], feed_dict={X: x_data, T: t_data})    
        #feed_dict의 값이 sess.run의 loss, y, train으로 입력이 되어서, 연산을 수행하고,
        #그 연산 값은 loss는 loss_val로, y값은 y_val로, train값은 _로 저장이 된다.
        if step % 400 == 0:
            print("step = ", step, ", loss_val = ", loss_val)             
            print("y_val값",y_val)
    print("\nPrediction is ", sess.run(y, feed_dict={X: [ [100, 98, 81] ]}))  

step =  0 , loss_val =  38147.19
y_val값 [[-31.861109]
 [-38.281754]
 [-38.76789 ]
 [-23.198404]
 [-10.348547]
 [-27.596048]
 [-19.020948]
 [-22.886286]
 [-13.084024]
 [-25.836508]
 [-18.561632]
 [-42.54071 ]
 [-38.5951  ]
 [-25.458673]
 [-30.612017]
 [-36.404182]
 [-34.532043]
 [-42.9526  ]
 [-39.07878 ]
 [-31.919489]
 [-27.592417]
 [-32.322876]
 [-44.16275 ]
 [-36.56846 ]]
step =  400 , loss_val =  28.696527
y_val값 [[187.7541  ]
 [178.99341 ]
 [197.52861 ]
 [144.1547  ]
 [112.32597 ]
 [149.25204 ]
 [112.093475]
 [181.56723 ]
 [175.40538 ]
 [144.3931  ]
 [148.42657 ]
 [182.2454  ]
 [148.63089 ]
 [151.91698 ]
 [192.0826  ]
 [141.79129 ]
 [177.87883 ]
 [171.65787 ]
 [153.06606 ]
 [175.34268 ]
 [178.21803 ]
 [165.52106 ]
 [142.19298 ]
 [191.45494 ]]
step =  800 , loss_val =  22.653318
y_val값 [[187.34259 ]
 [179.35364 ]
 [197.64412 ]
 [143.7107  ]
 [111.40267 ]
 [149.33112 ]
 [112.184494]
 [180.50983 ]
 [173.68494 ]
 [144.32375 ]
 [147.61581 ]
 [182.87518 ]
 [149.45251 ]
 [151.7504  ]
 [19

In [44]:
y.shape

TensorShape([Dimension(None), Dimension(1)])